In [1]:
import json
import pandas as pd

In [2]:
hdata_oririnal = pd.read_csv('C:\\jupyter\\statistic\\horse_data.csv', header = None, na_values = '?')
hdata = hdata_oririnal.loc[:,[0,1,3,4,5,6,10,22]]
hdata.rename(columns={0:'surgery?', 1:'Age', 3:'rectal temperature',\
                               4:'pulse',5:'raspiratory rate',6:'temperature of extremities',\
                               10:'pain',22:'outcome'}, inplace = True)

In [3]:
hdata

,surgery?,Age,rectal temperature,pulse,raspiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...
295,1.0,1,NaN,120.0,70.0,4.0,2.0,3.0
296,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
297,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


In [4]:
#находим моду
hdata.mode()

,surgery?,Age,rectal temperature,pulse,raspiratory rate,temperature of extremities,pain,outcome
0,1.0,1,38.0,48.0,20.0,3.0,3.0,1.0


In [5]:
#описываем весь массив при помощи функции describe
hdata.describe()

,surgery?,Age,rectal temperature,pulse,raspiratory rate,temperature of extremities,pain,outcome
count,299.000000,300.000000,240.000000,276.000000,242.000000,244.000000,245.00000,299.000000
mean,1.397993,1.640000,38.167917,71.913043,30.417355,2.348361,2.95102,1.551839
std,0.490305,2.173972,0.732289,28.630557,17.642231,1.045054,1.30794,0.737187
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.00000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.500000,1.000000,2.00000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,24.500000,3.000000,3.00000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.00000,2.000000
max,2.000000,9.000000,40.800000,184.000000,96.000000,4.000000,5.00000,3.000000


In [6]:
#считаем количество пропусков в каждом стоблце
hdata.isnull().sum()

surgery?                       1
Age                            0
rectal temperature            60
pulse                         24
raspiratory rate              58
temperature of extremities    56
pain                          55
outcome                        1
dtype: int64

In [7]:
#Предположим что строки содержащие 4 или 5 пропущенных значений (больше половины) являются недостоверными и удалим их
hdata = hdata.drop(hdata[(hdata.isnull().sum(axis = 1) == 5) | (hdata.isnull().sum(axis = 1) == 4)].index)
hdata

,surgery?,Age,rectal temperature,pulse,raspiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...
295,1.0,1,NaN,120.0,70.0,4.0,2.0,3.0
296,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
297,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


#### 1. Работаем с первым столбцом 'surgery?'

In [8]:
#Проверяем наличие битых значений в столбце 'surgery?' По условию д.б. только = 1 или = 2
hdata.loc[(hdata['surgery?'] != 1) & (hdata['surgery?'] != 2)]['surgery?'].count()

0

In [11]:
#находим пустое значение
hdata[hdata['surgery?'].isnull()]

,surgery?,Age,rectal temperature,pulse,raspiratory rate,temperature of extremities,pain,outcome


In [10]:
#запонлим пропуски в столбце
#предположим, что для молодой лошади, с нормальными значениями температуры, пульса и частоты дыхания операция не нужна
#если какое-либо значение не в норме, то операция нужна
hdata.loc[(hdata['surgery?'].isnull()) & (hdata['Age'] == 2) & (hdata['rectal temperature'] == 37.8) \
         & (hdata['pulse'].between(30,40)) & (hdata['raspiratory rate'].between(8, 10)), 'surgery?'] = 2
hdata.loc[(hdata['surgery?'].isnull()), 'surgery?'] = 1

#### 2. Работаем со вторым столбцом 'Age'

In [12]:
#Проверяем наличие битых значений в столбце. По условию д.б. только = 1 или = 2
print(hdata.loc[(hdata['Age'] != 1) & (hdata['Age'] != 2)]['Age'].count())
#находим недостоверные значения
hdata.loc[(hdata['Age'] != 1) & (hdata['Age'] != 2)]

22


,surgery?,Age,rectal temperature,pulse,raspiratory rate,temperature of extremities,pain,outcome
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
9,2.0,9,38.3,90.0,NaN,1.0,5.0,1.0
13,2.0,9,38.0,92.0,28.0,1.0,1.0,2.0
16,1.0,9,NaN,128.0,36.0,3.0,4.0,2.0
23,1.0,9,38.3,130.0,60.0,NaN,2.0,1.0
39,1.0,9,39.2,146.0,96.0,NaN,NaN,2.0
41,2.0,9,39.0,150.0,72.0,NaN,NaN,1.0
55,1.0,9,38.6,160.0,20.0,3.0,3.0,2.0
75,1.0,9,39.7,100.0,NaN,3.0,2.0,3.0
82,1.0,9,38.1,100.0,80.0,3.0,3.0,1.0


In [13]:
#Вернулось множество значений '9'. 
#Предположим, что случайно записали реальный возраст лошади (9 лет или 9 месяцев), заменим все значения 9 на 1 (Adult horse)
hdata.loc[hdata['Age'] == 9, 'Age'] = 1
# и проверяем
hdata.loc[(hdata['Age'] != 1) & (hdata['Age'] != 2)]['Age'].count()

0

#### 3. Работаем с третьим столбцом 'rectal temperature'

In [14]:
#Проверяем наличие битых значений в столбце. Температура тела лошади может быть от 35 до 41 градусов
hdata.loc[(hdata['rectal temperature'] < 35) | (hdata['rectal temperature'] > 41)]['rectal temperature'].count()

0

In [16]:
#находим пустые значения
hdata[hdata['rectal temperature'].isnull()].head(5)

,surgery?,Age,rectal temperature,pulse,raspiratory rate,temperature of extremities,pain,outcome
5,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0
7,1.0,1,NaN,60.0,NaN,3.0,NaN,2.0
8,2.0,1,NaN,80.0,36.0,3.0,4.0,3.0
16,1.0,1,NaN,128.0,36.0,3.0,4.0,2.0
34,1.0,1,NaN,100.0,30.0,3.0,5.0,1.0


In [17]:
#предположим, что rectal temperature связана с temperature of extremities (1 = Normal 2 = Warm 3 = Cool 4 = Cold)
#и будем присваивать при 
#temperature of extremities = 1 --> rectal temperature = как средняя температура у всех других лошадей с т.конечностей = 1
#temperature of extremities = 2 --> rectal temperature = аналогично, только т.конечностей = 2
#temperature of extremities = 3 --> rectal temperature =  аналогично, только т.конечностей = 3
#temperature of extremities = 4 --> rectal temperature =  аналогично, только т.конечностей = 4
hdata.loc[(hdata['rectal temperature'].isnull()) & (hdata['temperature of extremities'] == 1),'rectal temperature'] \
            =hdata[hdata['temperature of extremities'] == 1]['rectal temperature'].mean()
hdata.loc[(hdata['rectal temperature'].isnull()) & (hdata['temperature of extremities'] == 2),'rectal temperature'] \
            =hdata[hdata['temperature of extremities'] == 2]['rectal temperature'].mean()
hdata.loc[(hdata['rectal temperature'].isnull()) & (hdata['temperature of extremities'] == 3),'rectal temperature'] \
            =hdata[hdata['temperature of extremities'] == 3]['rectal temperature'].mean()
hdata.loc[(hdata['rectal temperature'].isnull()) & (hdata['temperature of extremities'] == 4),'rectal temperature'] \
            =hdata[hdata['temperature of extremities'] == 4]['rectal temperature'].mean()
#остались значения в которых и 'temperature of extremities' и 'rectal temperature' = NaN, в таком случае заполним их средним по 
# 'rectal temperature'
hdata.loc[(hdata['rectal temperature'].isnull()), 'rectal temperature'] = hdata['rectal temperature'].mean()

#### 4. Работаем с четвертым столбцом 'pulse'

In [18]:
#Проверяем наличие битых значений в столбце. Пульслошади может быть от 20 до 250 ударов в минуту
hdata.loc[(hdata['pulse'] < 20) | (hdata['pulse'] > 250)]['pulse'].count()

0

In [20]:
#находим пустые значения
hdata[hdata['pulse'].isnull()].head(5)

,surgery?,Age,rectal temperature,pulse,raspiratory rate,temperature of extremities,pain,outcome
5,2.0,1,38.203846,NaN,NaN,2.0,2.0,1.0
52,2.0,1,38.126389,NaN,NaN,1.0,1.0,1.0
58,1.0,1,38.455000,NaN,20.0,4.0,5.0,2.0
78,1.0,1,38.187500,NaN,NaN,3.0,5.0,2.0
83,1.0,1,38.000000,NaN,24.0,3.0,5.0,2.0


In [21]:
#предположим что пульс имеет среднее значение
hdata.loc[(hdata['pulse'].isnull()), 'pulse'] = hdata['pulse'].mean()

#### 5. Работаем с пятым столбцом 'raspiratory rate'

In [22]:
#Проверяем наличие битых значений в столбце. Частота дыхания лошади может быть от 5 до 200 вдохов в минуту
hdata.loc[(hdata['raspiratory rate'] < 5) | (hdata['raspiratory rate'] > 200)]['raspiratory rate'].count()

0

In [41]:
#находим пустые значения
hdata[hdata['raspiratory rate'].isnull()].head(5)

,surgery?,Age,rectal temperature,pulse,raspiratory rate,temperature of extremities,pain,outcome


In [40]:
#предположим что частота дыхания связана с пульсом, выделим три квантиля пульса
#первый до 48 --> присвоим частоте дыхания среднее значение частоты дыхания всех лошадей с пульсом до 48
#второй до 66--> присвоим частоте дыхания среднее значение частоты дыхания всех лошадей с пульсом до 66
#третий до 88--> присвоим частоте дыхания среднее значение частоты дыхания всех лошадей с пульсом до 88
hdata.loc[(hdata['raspiratory rate'].isnull()) & (hdata['pulse'].between(0,48)), 'raspiratory rate'] = \
            hdata[hdata['pulse'].between(0,48)]['raspiratory rate'].mean()
hdata.loc[(hdata['raspiratory rate'].isnull()) & (hdata['pulse'].between(49,66)), 'raspiratory rate'] = \
            hdata[hdata['pulse'].between(49,66)]['raspiratory rate'].mean()
hdata.loc[(hdata['raspiratory rate'].isnull()) & (hdata['pulse'].between(67,250)), 'raspiratory rate'] = \
            hdata[hdata['pulse'].between(67,250)]['raspiratory rate'].mean()  

#### 6. Работаем с шестым столбцом 'temperature of extremities'

In [25]:
#Проверяем наличие битых значений в столбце. temperature of extremities = 1 или 2 или 3 или 4
hdata.loc[(hdata['temperature of extremities'] != 1) & (hdata['temperature of extremities'] != 2) &\
          (hdata['temperature of extremities'] != 3) & (hdata['temperature of extremities'] != 4)] \
         ['temperature of extremities'].count()

0

In [27]:
#находим пустые значения
hdata[hdata['temperature of extremities'].isnull()].head(5)

,surgery?,Age,rectal temperature,pulse,raspiratory rate,temperature of extremities,pain,outcome
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
17,2.0,1,37.5,48.0,24.0,NaN,NaN,1.0
23,1.0,1,38.3,130.0,60.0,NaN,2.0,1.0
25,2.0,1,37.8,60.0,42.0,NaN,NaN,1.0


In [28]:
#сделаем предположением обратное столбцу 3
#предположим, что rectal temperature связана с temperature of extremities (1 = Normal 2 = Warm 3 = Cool 4 = Cold)
#и будем присваивать при 
# rectal temperature = 37.8 --> temperature of extremities = 1
# rectal temperature > 37.8 --> temperature of extremities = 2
# 37 < rectal temperature < 37.8 --> temperature of extremities = 3
# rectal temperature < 37.0 --> temperature of extremities = 4
hdata.loc[(hdata['temperature of extremities'].isnull()) & (hdata['rectal temperature'] == 37.8), \
          'temperature of extremities'] = 1
hdata.loc[(hdata['temperature of extremities'].isnull()) & (hdata['rectal temperature'] > 37.8), \
          'temperature of extremities'] = 2
hdata.loc[(hdata['temperature of extremities'].isnull()) & (hdata['rectal temperature'].between(37,37.8)), \
          'temperature of extremities'] = 3
hdata.loc[(hdata['temperature of extremities'].isnull()) & (hdata['rectal temperature'] < 37.0), \
          'temperature of extremities'] = 4

#### 7. Работаем с седьмым столбцом 'pain'

In [29]:
#Проверяем наличие битых значений в столбце. pain = 1 или 2 или 3 или 4 или 5
hdata.loc[(hdata['pain'] != 1) & (hdata['pain'] != 2) & (hdata['pain'] != 3) & (hdata['pain'] != 4) &\
         (hdata['pain'] != 5)]['pain'].count()

0

In [31]:
#находим пустые значения
hdata[hdata['pain'].isnull()].head(5)

,surgery?,Age,rectal temperature,pulse,raspiratory rate,temperature of extremities,pain,outcome
4,2.0,1,37.3000,104.0,35.0,3.0,NaN,2.0
7,1.0,1,38.1875,60.0,NaN,3.0,NaN,2.0
17,2.0,1,37.5000,48.0,24.0,3.0,NaN,1.0
19,2.0,1,39.4000,110.0,35.0,4.0,NaN,1.0
24,1.0,1,38.1000,60.0,12.0,3.0,NaN,1.0


In [32]:
#преположим, если лошади нужна surgery (= 1) то она имеет уровень боли 4 или 5, примем значение 4
#если не нужна surgery? (=2) то уровень боли 1 или 2, примем значение 2

hdata.loc[(hdata['pain'].isnull()) & (hdata['surgery?'] == 1), 'pain'] = 4
hdata.loc[(hdata['pain'].isnull()) & (hdata['surgery?'] == 2), 'pain'] = 2

#### 8. Работаем с восьмым столбцом 'outcome'

In [33]:
#Проверяем наличие битых значений в столбце. outcome = 1 или 2 или 3
hdata.loc[(hdata['outcome'] != 1) & (hdata['outcome'] != 2) & (hdata['outcome'] != 3)]['outcome'].count()

0

In [34]:
#находим пустые значения
hdata[hdata['outcome'].isnull()]

,surgery?,Age,rectal temperature,pulse,raspiratory rate,temperature of extremities,pain,outcome
132,1.0,1,38.0,48.0,20.0,3.0,4.0,NaN


In [35]:
#предположим что лошадь осталась жива (=1) если ей не требовалась операция (=2) и она была молодой (=2)
#в иных случаях она умирает (=2)

hdata.loc[(hdata['outcome'].isnull()) & (hdata['surgery?'] == 2) & (hdata['Age'] == 2), 'outcome'] = 1
hdata.loc[hdata['outcome'].isnull(), 'outcome'] = 2

In [36]:
#Таким образом, получаем массив без пропусков
hdata

,surgery?,Age,rectal temperature,pulse,raspiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.500,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.200,88.0,20.0,2.0,3.0,3.0
2,2.0,1,38.300,40.0,24.0,1.0,3.0,1.0
3,1.0,1,39.100,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.300,104.0,35.0,3.0,2.0,2.0
...,...,...,...,...,...,...,...,...
295,1.0,1,38.455,120.0,70.0,4.0,2.0,3.0
296,2.0,1,37.200,72.0,24.0,3.0,4.0,3.0
297,1.0,1,37.500,72.0,30.0,4.0,4.0,2.0
298,1.0,1,36.500,100.0,24.0,3.0,3.0,1.0


In [43]:
#последняя проверка на пропущенные значения
hdata.isnull().sum()

surgery?                      0
Age                           0
rectal temperature            0
pulse                         0
raspiratory rate              0
temperature of extremities    0
pain                          0
outcome                       0
dtype: int64

## Условия:
#### surgery?
- 1 = Yes, it had surgery
- 2 = It was treated without surgery
#### Age
- 1 = Adult horse
- 2 = Young (< 6 months)
#### Reactal Temperature
- in degrees celsius.
- An elevated temp may occur due to infection.
- temperature may be reduced when the animal is in late shock
- normal temp is 37.8
- this parameter will usually change as the problem progresses eg. may start out normal, then become elevated because of       the lesion, passing back through the normal range as the horse goes into shock
#### Pulse
- linear
- the heart rate in beats per minute
- is a reflection of the heart condition: 30 -40 is normal for adults
- rare to have a lower than normal rate although athletic horses may have a rate of 20-25
- animals with painful lesions or suffering from circulatory shock may have an elevated heart rate
#### raspiratory rate
- linear
- normal rate is 8 to 10
- usefulness is doubtful due to the great fluctuations
#### temperature of extremities
- a subjective indication of peripheral circulation
- possible values:
     1 = Normal
     2 = Warm
     3 = Cool
     4 = Cold
- cool to cold extremities indicate possible shock
- hot extremities should correlate with an elevated rectal temp.
#### pain
- a subjective judgement of the horse's pain level
- possible values:
    1 = alert, no pain
    2 = depressed
    3 = intermittent mild pain
    4 = intermittent severe pain
    5 = continuous severe pain
- should NOT be treated as a ordered or discrete variable!
- In general, the more painful, the more likely it is to require surgery
- prior treatment of pain may mask the pain level to some extent
#### outcome
- what eventually happened to the horse?
- possible values:
    1 = lived
    2 = died
    3 = was euthanized